In [1]:
import getpass
import os
import imghdr
from PIL import Image

In [2]:
# get the username
uname=getpass.getuser()

In [3]:
# get the spotlight folder 
slPathpart="AppData\\Local\\Packages\\Microsoft.Windows.ContentDeliveryManager_cw5n1h2txyewy\\LocalState\\Assets"
slPath=os.path.join('C:\\Users', uname, slPathpart)
os.chdir(slPath)

In [4]:
# create the SpotlightImagesRevealed user folder
slPathUser=os.path.join('C:\\Users', uname, 'Pictures\\SpotlightImagesRevealed')
if not os.path.exists(slPathUser):
    os.makedirs(slPathUser)

In [5]:
# delete all files in SpotLightRevealed folder
for the_file in os.listdir(slPathUser):
    file_path = os.path.join(slPathUser, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as e:
        print(e)

In [6]:
# copy all files from SpotLight source folder to SpotlightImagesRevealed one
from distutils.dir_util import copy_tree
fromDirectory = slPath
toDirectory = slPathUser
copy_tree(fromDirectory, toDirectory)

['C:\\Users\\andybandy\\Pictures\\SpotlightImagesRevealed\\040cabc8d2c98d2ffcaed125b7c3a0f41f9cec1304b4c01546ba51561a590178',
 'C:\\Users\\andybandy\\Pictures\\SpotlightImagesRevealed\\04b5fc303d11bb6650bf488cf44a1e9bedd192152f205cfb3f30f388a7fcb347',
 'C:\\Users\\andybandy\\Pictures\\SpotlightImagesRevealed\\06e51e3d3d0646064c03f083270df556b96f562cad06085ea73a9a4ce1b509c6',
 'C:\\Users\\andybandy\\Pictures\\SpotlightImagesRevealed\\0b5ed0fcb47e99eb9e453a4840b868559050b844e04d90a2b63d39af1e8e183b',
 'C:\\Users\\andybandy\\Pictures\\SpotlightImagesRevealed\\0c0eede1394421e9368ca381b006710a6eeaa6f59cfac0e2710847e01b6b9dec',
 'C:\\Users\\andybandy\\Pictures\\SpotlightImagesRevealed\\0cb49d8e3c5248f98e75898e66fdaecbf24722528cdd5c9be08b8647b6c42ac8',
 'C:\\Users\\andybandy\\Pictures\\SpotlightImagesRevealed\\0dc930d390a2f85baba4dfc96a00a59a5f0472a34d687d8e1d64a5a29dd44aa5',
 'C:\\Users\\andybandy\\Pictures\\SpotlightImagesRevealed\\0df771c74868690fc305d62660a0b99bbc89c57e69989b62dcfab43b74e

In [7]:
# delete all files that are not jpeg
for root, _, files in os.walk(slPathUser):
    for f in files:
        fullpath = os.path.join(root, f)
        imageType=imghdr.what(fullpath)
        try:
            if (imageType != 'jpeg'):
                os.remove(fullpath)
        except WindowsError:
            print("Error " + fullpath)

In [8]:
# add the right extension to files
for root, _, files in os.walk(slPathUser):
    for f in files:
        fullpath = os.path.join(root, f)
        imageType=imghdr.what(fullpath)
        try:
            suffix = imageType
            newName = os.path.join(fullpath + '.' + suffix)
            os.rename(fullpath,newName)
        except WindowsError:
            print("Error " + fullpath)

In [9]:
# remove the images that have a portrait ratio or a widht lower than 1920px
for root, _, files in os.walk(slPathUser):
    for f in files:
        fullpath = os.path.join(root, f)
        im = Image.open(fullpath)
        width, height = im.size
        ratio = width/height
        im.close()
        if (ratio <= 1.0) or (width < 1920):
            try:
                os.remove(fullpath)
            except WindowsError:
                print("Error " + fullpath)